In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_192c_J8_nf8_v1.0.2.pth

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [11]:
import torch
import numpy as np
import torchaudio
import tempfile
from demucs.separate import Separator
from IPython.display import Audio
from datasets import load_dataset
from walloc import walloc
from spauq.core.metrics import spauq_eval
class Config: pass

In [3]:
device = "cuda"
checkpoint = torch.load("Stereo_Li_192c_J8_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
).to(device)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [4]:
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [7]:
sources = ['other', 'drums', 'bass', 'vocals']
separator = Separator()

In [8]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [31]:
max_duration=44100*180
with torch.no_grad():
    for i_sample in range(0, len(MUSDB), 5):
        y = []
        for i_instr in range(5):
            sample = MUSDB[i_sample+i_instr]
            instr = sample['instrument']
            x, fs = torchaudio.load(sample['audio']['bytes'])
            x = x[:,:max_duration]
            L = x.shape[-1]
            x_padded = pad(x.unsqueeze(0), 2**16).to(device)
            if i_instr == 0:
                pred = separator.separate_tensor(x_padded[0])
            else:
                SDR = spauq_eval(
                    reference=x_padded[0].to("cpu"),
                    estimate=pred[1][instr].to("cpu"),
                    fs = fs
                )
                break
        break

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [32]:
SDR

{'SSR': 12.12786068375291, 'SRR': 5.170535327707984}